In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/seattle/calendar.csv
/kaggle/input/seattle/listings.csv
/kaggle/input/seattle/reviews.csv


In [2]:
seattle_calendar = pd.read_csv("/kaggle/input/seattle/calendar.csv")
seattle_listings = pd.read_csv("/kaggle/input/seattle/listings.csv")
seattle_reviews = pd.read_csv("/kaggle/input/seattle/reviews.csv")

Just taking a quick look at what the dataframes look like

In [3]:
seattle_calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [4]:
seattle_calendar.describe()

,listing_id
count,1.393570e+06
mean,5.550111e+06
std,2.962274e+06
min,3.335000e+03
25%,3.258213e+06
50%,6.118244e+06
75%,8.035212e+06
max,1.034016e+07


In [5]:
seattle_listings.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


In [6]:
seattle_listings.describe()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,...,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,reviews_per_month
count,3.818000e+03,3.818000e+03,3.818000e+03,3816.000000,3816.000000,3818.000000,3818.000000,3818.000000,3802.000000,3812.000000,...,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3162.000000,0.0,3818.000000,3191.000000
mean,5.550111e+06,2.016010e+13,1.578556e+07,7.157757,7.157757,47.628961,-122.333103,3.349398,1.259469,1.307712,...,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,9.452245,NaN,2.946307,2.078919
std,2.962660e+06,0.000000e+00,1.458382e+07,28.628149,28.628149,0.043052,0.031745,1.977599,0.590369,0.883395,...,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,0.750259,NaN,5.893029,1.822348
min,3.335000e+03,2.016010e+13,4.193000e+03,1.000000,1.000000,47.505088,-122.417219,1.000000,0.000000,0.000000,...,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,2.000000,NaN,1.000000,0.020000
25%,3.258256e+06,2.016010e+13,3.275204e+06,1.000000,1.000000,47.609418,-122.354320,2.000000,1.000000,1.000000,...,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,9.000000,NaN,1.000000,0.695000
50%,6.118244e+06,2.016010e+13,1.055814e+07,1.000000,1.000000,47.623601,-122.328874,3.000000,1.000000,1.000000,...,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,1.000000,1.540000
75%,8.035127e+06,2.016010e+13,2.590309e+07,3.000000,3.000000,47.662694,-122.310800,4.000000,1.000000,2.000000,...,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,2.000000,3.000000
max,1.034016e+07,2.016010e+13,5.320861e+07,502.000000,502.000000,47.733358,-122.240607,16.000000,8.000000,7.000000,...,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,NaN,37.000000,12.150000


In [7]:
seattle_reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [8]:
seattle_reviews.describe()

,listing_id,id,reviewer_id
count,8.484900e+04,8.484900e+04,8.484900e+04
mean,3.005067e+06,3.058765e+07,1.701301e+07
std,2.472877e+06,1.636613e+07,1.353704e+07
min,4.291000e+03,3.721000e+03,1.500000e+01
25%,7.946330e+05,1.725127e+07,5.053141e+06
50%,2.488228e+06,3.228809e+07,1.413476e+07
75%,4.694479e+06,4.457648e+07,2.762402e+07
max,1.024814e+07,5.873651e+07,5.281274e+07


In [9]:
seattle_calendar.columns.values

array(['listing_id', 'date', 'available', 'price'], dtype=object)

In [10]:
seattle_listings.columns.values

array(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name',
       'summary', 'space', 'description', 'experiences_offered',
       'neighborhood_overview', 'notes', 'transit', 'thumbnail_url',
       'medium_url', 'picture_url', 'xl_picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location',
       'host_about', 'host_response_time', 'host_response_rate',
       'host_acceptance_rate', 'host_is_superhost', 'host_thumbnail_url',
       'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode',
       'market', 'smart_location', 'country_code', 'country', 'latitude',
       'longitude', 'is_location_exact', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       

In [11]:
seattle_reviews.columns.values

array(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name',
       'comments'], dtype=object)

In [12]:
#figuring out what each of the features in seattle_listings means
for feature in seattle_listings.columns.values:
    print(seattle_listings[feature].head())
    print("")

0     241032
1     953595
2    3308979
3    7421966
4     278830
Name: id, dtype: int64

0     https://www.airbnb.com/rooms/241032
1     https://www.airbnb.com/rooms/953595
2    https://www.airbnb.com/rooms/3308979
3    https://www.airbnb.com/rooms/7421966
4     https://www.airbnb.com/rooms/278830
Name: listing_url, dtype: object

0    20160104002432
1    20160104002432
2    20160104002432
3    20160104002432
4    20160104002432
Name: scrape_id, dtype: int64

0    2016-01-04
1    2016-01-04
2    2016-01-04
3    2016-01-04
4    2016-01-04
Name: last_scraped, dtype: object

0           Stylish Queen Anne Apartment
1     Bright & Airy Queen Anne Apartment
2    New Modern House-Amazing water view
3                     Queen Anne Chateau
4         Charming craftsman 3 bdm house
Name: name, dtype: object

0                                                  NaN
1    Chemically sensitive? We've removed the irrita...
2    New modern house built in 2013.  Spectacular s...
3    A charming apartmen

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


It might be possible to make some simple categorical features out of the text columns, such as whether certain words appear in the description or review for example, so I will keep those for now. Selected features that could be relevant for predicting the label (price)

In [13]:
#keeping features that may either be of use in determining duplicates or for the
#actual price prediction
selected_listing_columns = ["last_review", "street", "name", "summary", "space", "description", 
                            "neighborhood_overview", "notes", "transit", "host_since", 
                           "host_about", "host_response_time", "host_response_rate", 
                           "host_is_superhost", "host_neighbourhood", "host_listings_count", 
                           "host_total_listings_count", "host_has_profile_pic", 
                           "host_identity_verified", "neighbourhood", "neighbourhood_cleansed", 
                           "neighbourhood_group_cleansed", "property_type", "room_type",
                           "accommodates", "bathrooms", "bedrooms", "beds", "bed_type",
                           "amenities", "square_feet", "review_scores_rating", 
                           "review_scores_accuracy", "review_scores_cleanliness", 
                           "review_scores_checkin", "review_scores_communication", 
                           "review_scores_location", "review_scores_value", "cancellation_policy", 
                           "calculated_host_listings_count", "price"]
new_seattle_listings = seattle_listings[selected_listing_columns].copy()
print("The new seattle listings dataframe will use", 
      new_seattle_listings.shape[1] - 1, "features, and the output label will be", 
     new_seattle_listings.columns[-1])

The new seattle listings dataframe will use 40 features, and the output label will be price


In [14]:
print(new_seattle_listings.shape)
print(new_seattle_listings['notes'][9])
print("")
print(new_seattle_listings['notes'][1])
print("")
print(new_seattle_listings['price'][1])
print("")
print(new_seattle_listings['price'][9])
print(new_seattle_listings['notes'][1] == new_seattle_listings['notes'][9])
print(new_seattle_listings['price'][1] == new_seattle_listings['price'][9])

(3818, 41)
What's up with the free pillows?  Our home was renovated and restored with health & safety in mind, especially the elimination/reduction of indoor allergens, toxins & other pollutants.  (For this reason, we have no carpets, just hardwood & slate floors, with radiant heat.)  Pillows tend to house a LOT of allergens over time, even when just used by a single owner.  The statistics make many people feel pretty queasy but, essentially, pillows become great breeding grounds for bacteria and dust mites, not to mention drool, hair oils and perspiration. Since most people sweat more than a pint a night, the pillow can also easily trap mildew and mold. We use dust mite protectors for mattresses and duvets, but haven't found any that work well for pillows that maintain comfort/elasticity without the use of outgassing materials.  So, I let the guests take their pillows but ask that they leave the pillowcases. And if you leave the pillows behind, they'll be donated to a local women's sh

Look at the entries above! It appears that there are duplicate entries in the dataframe. We want to remove these duplicates so that every entry is unique.

In [15]:
#Remove duplicate entries. Here we are just removing exact duplicates, which could be a
#limitation if there are near duplicates but I am just handling this for now
print("Shape of original dataframe (with features selected):",new_seattle_listings.shape)
exact_duplicates_dropped = new_seattle_listings.drop_duplicates()
print("Shape after dropping exact duplicates:",exact_duplicates_dropped.shape)
subset_dropped_listings_np = new_seattle_listings.drop_duplicates(subset=['notes', 'price'])
print("Shape after dropping duplicates with exact match of notes and price:",
      subset_dropped_listings_np.shape)
subset_dropped_listings_notes = new_seattle_listings.drop_duplicates(subset=['notes'])
print("Shape after dropping duplicates with exact match of notes:",
      subset_dropped_listings_notes.shape)

#trying a subset of house-related stuff, maybe this will reveal duplicates accurately?
#not sure 
subset_dropped_listings_house_features = new_seattle_listings.drop_duplicates(
    subset=['property_type', 'bathrooms', 'bedrooms', 'square_feet', 'neighbourhood'])
print("Shape after dropping duplicates with matching house related features:", 
     subset_dropped_listings_house_features.shape)
subset_dropped_listings_name = new_seattle_listings.drop_duplicates(
    subset=['name']
)
print("Shape after dropping duplicates with exact match of name: ",
     subset_dropped_listings_name.shape)

Shape of original dataframe (with features selected): (3818, 41)
Shape after dropping exact duplicates: (3818, 41)
Shape after dropping duplicates with exact match of notes and price: (2337, 41)
Shape after dropping duplicates with exact match of notes: (2000, 41)
Shape after dropping duplicates with matching house related features: (1071, 41)
Shape after dropping duplicates with exact match of name:  (3792, 41)


As we can see from some experimentation dropping duplicates, there are no exact duplicates! However, if we limit to dropping if the exact price and notes are the same, we get a significant reduction in entries. We reduce even more entries if we just look for if the notes are the same. This suggestions, for example, that someone may have put the same listing up but with different prices, perhaps due to increased demand or they weren't able to get anyone to book so they decreased the price. Also, it's possible that some entries are duplicates but won't be caught using notes and price as the subset, but will be caught using a different subset. We cannot try all the subsets of the features that we have, so we will need an effective method to identify duplicates entries.

In [16]:
#I need to see what the full listings look like for some of these examples that have a 
#couple duplicate features, for example notes and price, look like. We know indices 1 and 9
#are potential duplicates

#print(new_seattle_listings.iloc[1])
print("")
#print(new_seattle_listings.iloc[9])
unequal_features = []
for feature in new_seattle_listings:
    equal_feature = (new_seattle_listings[feature][1] == new_seattle_listings[feature][9])
    print(feature, equal_feature)
    if not equal_feature:
        unequal_features.append(feature)
print("----------------")

#these two neighborhood_overview look exactly the same so printing out length to see why they
#are unequal
print(len(new_seattle_listings["neighborhood_overview"][1]))
print(len(new_seattle_listings["neighborhood_overview"][9]))
for f in unequal_features:
    print(f, "for listing 1:", new_seattle_listings[f][1])
    print("")
    print(f, "for listing 9:", new_seattle_listings[f][9])
    print("")


last_review False
street True
name False
summary False
space False
description False
neighborhood_overview False
notes True
transit True
host_since True
host_about True
host_response_time True
host_response_rate True
host_is_superhost True
host_neighbourhood True
host_listings_count True
host_total_listings_count True
host_has_profile_pic True
host_identity_verified True
neighbourhood True
neighbourhood_cleansed True
neighbourhood_group_cleansed True
property_type True
room_type True
accommodates True
bathrooms True
bedrooms True
beds True
bed_type True
amenities True
square_feet False
review_scores_rating False
review_scores_accuracy True
review_scores_cleanliness True
review_scores_checkin True
review_scores_communication True
review_scores_location True
review_scores_value False
cancellation_policy True
calculated_host_listings_count True
price True
----------------
167
169
last_review for listing 1: 2015-12-29

last_review for listing 9: 2015-12-20

name for listing 1: Bright & Ai

These examples above are clearly duplicates. The only thing I see that's different is the review_scores_rating. This makes sense as these could have been scraped at different times and reviews could have been added, thus changing the rating. Clearly the same property though. My intuition is that using stuff that cannot change about a property, such as bedrooms, and bathrooms, room type (I would say someone listing the entire house vs. listing one room even if it's the same house constitutes a different listing), etc. is good to determine duplicates. But how many checks are required? Two different houses could both be 2bed/2bath, for example.  

In [17]:
subset_dropped_listings_street = new_seattle_listings.drop_duplicates(
    subset=['street']
)
print("Shape after dropping duplicates with exact match of street: ",
     subset_dropped_listings_street.shape)
subset_dropped_listings_sbd = new_seattle_listings.drop_duplicates(
    subset=['street', 'room_type', 'bedrooms', 'bathrooms']
)
print("Shape after dropping duplicates with exact match of street and checking \
for the same listing type within the same address: ",
     subset_dropped_listings_sbd.shape)

Shape after dropping duplicates with exact match of street:  (1442, 41)
Shape after dropping duplicates with exact match of street and checking for the same listing type within the same address:  (2621, 41)


I didn't think of this earlier but as seen here, there are a significant number of listings with the same street address. Removing those takes us from 3818 entries down to 1442 entries. However, it is possible that two listings could be on the same address but actually different properties or listing types. For example, For this simple check, I am happy to use this result, and just figure out how to keep the most recent listing with this result. In the new dataframe, I will also remove this street feature as it shouldn't influence the price.   

In [18]:
print("latest review for listing 1:", new_seattle_listings["last_review"][1])
print("latest review for listing 9:", new_seattle_listings["last_review"][9])

latest review for listing 1: 2015-12-29
latest review for listing 9: 2015-12-20


Just based on the above cell, comparing the last review dates for these two known duplicates (because the scrape date was the same for every entry so it doesn't provide any information), I will assume the most recent entries are at the top of the dataframe. Drop duplicates by default keeps the top entries so no modifications needed if proceeding based on this assumption. It's not time to assemble the actual dataframe based on 

In [19]:
#dropping out features that I am pretty sure will not be so useful or will be hard to 
#represent in a way that can train linear regression. I will maybe train another model that 
#attempts to represent features like description in a convoluted way and see if 
#it adds anything. Square feet is being dropped because almost all the values are missing
columns_to_drop = ["last_review", "street", "name", "summary", "space", "description", 
                  "neighborhood_overview", "notes", "host_about", "host_total_listings_count",
                  "host_has_profile_pic", "square_feet", "calculated_host_listings_count"]
deduplicated_listing_df = subset_dropped_listings_sbd.drop(columns=columns_to_drop)
#re-indexing to make sure there are not gaps when we try to key by index
deduplicated_listing_df = deduplicated_listing_df.reset_index(drop=True)
print(deduplicated_listing_df.shape)

(2621, 28)


Now it's time to represent all the features numerically. Steps:
1) Represent categorical features (such as description or needs_verification) as a 
vector. This needs to be addressed on a feature by feature basis as well, for example use
one-hot encoding for something vs. a different method. As an example,
my thought for amenities is to find all the possible amenities, then have a binary feature for whether a house has each one. 
2) Impute any missing values for features that are already numerical.. Figure out feature 
by feature what the most logical way to do this is. It's possible that some entire 
examples will need to be removed.
3) Scaling and normalization.
4) I might need to remove some features because they either a) have too many missing values or b) are not feasible to easily represent as a number or a vector of numbers
5) Address outliers.


In [20]:
#Step 1: Convert categorial features to numbers
#Need to figure out what to do for each feature

print("List of features: ")
print(deduplicated_listing_df.columns.to_list())
print("-----------------")


List of features: 
['transit', 'host_since', 'host_response_time', 'host_response_rate', 'host_is_superhost', 'host_neighbourhood', 'host_listings_count', 'host_identity_verified', 'neighbourhood', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'cancellation_policy', 'price']
-----------------


Above, listing out the features again for convenience. We can print out all the possible values for transit to get a sense of the descriptions (below). Based on what I saw, the non-null descriptions all describe convenient access to some transit. Thus, I can assume non-null values have transit, and null values do not have transit. However, another approach I could try (I think I will do this) is finding some key words that relate to different forms of transit (bus, car, taxi/uber, walking, ferry, etc) and encode each of these as a binary variable (convenient_bus_access, convenient_car_access, etc)

In [21]:
#Figure out what columns actually have missing values and how many. When we actually impute
#missing values, we will do it one feature at a time after transforming the feature to 
#numeric values if needed

missing_values = [feature for feature in deduplicated_listing_df if 
                  deduplicated_listing_df[feature].isna().any()]
#Find out how many missing values each column has
missing_value_counts = []
for feature in missing_values:
    missing_value_counts.append(deduplicated_listing_df[feature].isna().sum())

#Convert to a map
missing_value_freq_map = {missing_values[i]: missing_value_counts[i] 
                          for i in range(len(missing_values))}
n_rows = deduplicated_listing_df.shape[0]
print(n_rows)
missing_value_percent_map = {feature: round(missing_value_freq_map[feature] / n_rows, 3)
                        for feature in missing_value_freq_map}
print("Occurences of missing values")
print("------------------------------------")
print(missing_value_freq_map)
print("------------------------------------")
print(missing_value_percent_map)

2621
Occurences of missing values
------------------------------------
{'transit': 637, 'host_since': 1, 'host_response_time': 381, 'host_response_rate': 381, 'host_is_superhost': 1, 'host_neighbourhood': 245, 'host_listings_count': 1, 'host_identity_verified': 1, 'neighbourhood': 334, 'property_type': 1, 'bathrooms': 12, 'bedrooms': 6, 'beds': 1, 'review_scores_rating': 464, 'review_scores_accuracy': 472, 'review_scores_cleanliness': 468, 'review_scores_checkin': 472, 'review_scores_communication': 468, 'review_scores_location': 471, 'review_scores_value': 472}
------------------------------------
{'transit': 0.243, 'host_since': 0.0, 'host_response_time': 0.145, 'host_response_rate': 0.145, 'host_is_superhost': 0.0, 'host_neighbourhood': 0.093, 'host_listings_count': 0.0, 'host_identity_verified': 0.0, 'neighbourhood': 0.127, 'property_type': 0.0, 'bathrooms': 0.005, 'bedrooms': 0.002, 'beds': 0.0, 'review_scores_rating': 0.177, 'review_scores_accuracy': 0.18, 'review_scores_cleanlin

In [22]:
#Getting a sense of transit
for idx in range(10):
    print("")
    print(deduplicated_listing_df['transit'][idx])


nan

Convenient bus stops are just down the block, if you're headed downtown or further afield to explore.  Zip cars are also strategically located on Queen Anne hill.

A bus stop is just 2 blocks away.   Easy bus access to all the sites: Lower Queen Anne, Seattle Center, The Space Needle, Experience Music Project, Chihuly Glass Museum, Pacific Science Center and more.

nan

The nearest public transit bus (D Line) is 2 blocks away next to the Whole Foods Market on 15th Ave. W. Downtown is a 15 minute bus ride or a 7 minute drive. The light rail located at Westlake Center in downtown runs to the airport. Parking: street parking is available - unlimited and no zoning

The #1 and #2 bus picks up 2-3 blocks away (different directions).  They both go downtown to Westlake.

The bus goes through the lower Queen Anne neighborhood past Seattle Center and to downtown.  From downtown you can catch the light rail to the airport or ride a ferry. It is harder to bike around our neighborhood because

In [23]:
#seeing how many listings have different types of transit close-by (based on popular keywords)
#to describe these forms of transportation
import re

keyword_map = {
    'bus': ['bus', 'line'],
    'car': ['car', 'drive', '99', '5', 'freeway', 'highway', 'hwy'],
    'ferry': ['ferry', 'boat'],
    'taxi': ['taxi', 'cab', 'Uber', 'Lyft'],
    'walk_bike': ['walk', 'bike', 'bicycle', 'foot'],
    'train': ['train', 'rail', 'subway', 'metro']
}

print("Overall number of listings:", n_rows)
for transit_type, keywords in keyword_map.items():
    pattern = "|".join(re.escape(word) for word in keywords)
    count = deduplicated_listing_df['transit'].str.contains(
        pattern, case=False, na=False
    ).sum()
    print("Number of listings with convenient {} access: {}".format(transit_type, count))

Overall number of listings: 2621
Number of listings with convenient bus access: 1737
Number of listings with convenient car access: 1152
Number of listings with convenient ferry access: 61
Number of listings with convenient taxi access: 580
Number of listings with convenient walk_bike access: 911
Number of listings with convenient train access: 711


This seems reasonable. Now that we have a good sense of how many of each type of transit 
the properties have, let's convert these to binary numeric features. 

In [24]:
import re

#The keys to the dictionary are the new feature names we are inserting into the dataframe
keyword_map = {
    'convenient_bus_access': ['bus', 'line'],
    'convenient_car_access': ['car', 'drive', '99', '5', 'freeway', 'highway', 'hwy'],
    'convenient_ferry_access': ['ferry', 'boat'],
    'convenient_taxi_access': ['taxi', 'cab', 'Uber', 'Lyft'],
    'convenient_walk_bike_access': ['walk', 'bike', 'bicycle', 'foot'],
    'convenient_train_access': ['train', 'rail', 'subway', 'metro']
}

#Adding new binary features (the ones in the keyword map). For example 1 if transit 
#description contains one of the keywords, 0 otherwise. Done using regex 
for new_feature, keywords in keyword_map.items():
    pattern = "|".join(re.escape(word) for word in keywords)
    deduplicated_listing_df[new_feature] = deduplicated_listing_df['transit'].str.contains(
        pattern, case=False, na=False).astype(int)

#Dropping the transit column
df_transit_dropped = deduplicated_listing_df.drop(columns=['transit'])
print(df_transit_dropped.shape)

#Testing to see if the values match from the previous cell
for feature in keyword_map:
    print("Listings with {}: {}".format(feature, df_transit_dropped[feature].sum()))

#print(df_transit_dropped['transit']) produces a key error, as expected

(2621, 33)
Listings with convenient_bus_access: 1737
Listings with convenient_car_access: 1152
Listings with convenient_ferry_access: 61
Listings with convenient_taxi_access: 580
Listings with convenient_walk_bike_access: 911
Listings with convenient_train_access: 711


OK. Looks like the change to the dataframe was successful because the values match those from above and transit is now gone from the dataframe.

In [25]:
#For variables of interest, take a look at what all the values look like just to see what
#type of stuff is missing. For example, looking at all summaries. Just for my purposes I don't
#need to keep this here permanently
pd.set_option('display.max_rows', None)
print(new_seattle_listings['notes'][9])

What's up with the free pillows?  Our home was renovated and restored with health & safety in mind, especially the elimination/reduction of indoor allergens, toxins & other pollutants.  (For this reason, we have no carpets, just hardwood & slate floors, with radiant heat.)  Pillows tend to house a LOT of allergens over time, even when just used by a single owner.  The statistics make many people feel pretty queasy but, essentially, pillows become great breeding grounds for bacteria and dust mites, not to mention drool, hair oils and perspiration. Since most people sweat more than a pint a night, the pillow can also easily trap mildew and mold. We use dust mite protectors for mattresses and duvets, but haven't found any that work well for pillows that maintain comfort/elasticity without the use of outgassing materials.  So, I let the guests take their pillows but ask that they leave the pillowcases. And if you leave the pillows behind, they'll be donated to a local women's shelter that 